In [1]:
from typing import List, Text, Tuple, Union
from pprint import pprint

from pyvi import ViTokenizer
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as LR
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, FunctionTransformer
from sklearn.multiclass import OneVsRestClassifier as OVRStrategy
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from emandai.utils import load_data_from_botid
from tqdm import tqdm
from bokeh.io import output_file, output_notebook, export_png, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge

from va.src.utils.data import load_va_data

# First Look

## Load Data

In [85]:
PATH = "../data/trungquan/trainset.xlsx"
X, y = load_va_data(PATH)

## Preprocess Text

In [86]:
def preprocess(texts):
    """ Word Segmentation """
    tokenize = lambda x: ViTokenizer.tokenize(x)

    return [tokenize(text) for text in texts]

In [87]:
X = preprocess(X)

## Encode Label

In [88]:
lb = LabelBinarizer()
y = lb.fit_transform(y)
pprint(lb.classes_)
pprint(y)

array(['C2A_TUVAN', 'C2B_GLS', 'C3_KQT'], dtype='<U9')
array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])


In [89]:
def encode_label(labels):
    lb = LabelBinarizer()
    lb.fit(labels)
    return lb

## Text Embedding

### TF-IDF

In [90]:
tfidf = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", ngram_range=(1, 2))

In [91]:
tfidf_X = tfidf.fit_transform(X)

In [92]:
len(tfidf.vocabulary_)

1281

### Vietnamese SBERT

In [229]:
sbert = SentenceTransformer('keepitreal/vietnamese-sbert')

In [93]:
sbert_X = sbert.encode(X)

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

In [94]:
sbert_X.shape

(396, 768)

### PhoBERT

In [331]:
phobert = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base")
photokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

In [95]:
def phobert_encode(text: Union[Text, List[Text]],
                   phobert,
                   photokenizer) -> np.ndarray:
    """ Get sentence embedding from [CLS] token of the given text.
    reference: https://discuss.huggingface.co/t/how-to-get-cls-embeddings-from-bertfortokenclassification-model/9276/3
    """
    embeddings = None

    encoded_text = photokenizer(text, padding=True, truncation=True, return_tensors="pt")
    text_ids = encoded_text["input_ids"]
    with torch.no_grad():
        outputs = phobert(text_ids, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]
        embeddings = last_hidden_state[:, 0, :]

        if isinstance(embeddings, torch.Tensor):
            embeddings = embeddings.detach().cpu().numpy()

        return embeddings

In [2]:
class PhoBertWrapper:
    def __init__(self):
        self.model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base")
        self.tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

    def encode(self, X: Union[Text, List[Text]]):
        embeddings = None

        encoded_text = self.tokenizer(X, padding=True, truncation=True, return_tensors="pt")
        text_ids = encoded_text["input_ids"]
        with torch.no_grad():
            outputs = self.model(text_ids, output_hidden_states=True)
            last_hidden_state = outputs.hidden_states[-1]
            embeddings = last_hidden_state[:, 0, :]
            if isinstance(embeddings, torch.Tensor):
                embeddings = embeddings.detach().cpu().numpy()

        return embeddings

In [ ]:
phobert_X = phobert_encode(X, phobert, photokenizer)

In [80]:
phobert_X.shape

(396, 768)

## Classifier

### Logistic Regression

In [ ]:
clf = {}
clf["tfidf"] = OneVsRestClassifier(LogisticRegression()).fit(tfidf_X, y)
clf["sbert"] = OneVsRestClassifier(LogisticRegression()).fit(sbert_X, y)
clf["phobert"] = OneVsRestClassifier(LogisticRegression()).fit(phobert_X, y)

In [77]:
""" Inference """
def infer(text: Text,
          embedding_type: Text = "tfidf",
          with_prob: bool = False
    ) -> Union[np.ndarray, Tuple[np.ndarray, np.ndarray]]:
    pred = []

    if not isinstance(text, List):
        text = [text]

    if embedding_type == "tfidf":
        vectorizer = tfidf
        _clf = clf["tfidf"]
    elif embedding_type == "sbert":
        vectorizer = sbert
        _clf = clf["sbert"]
    elif embedding_type == "phobert":
        vectorizer = phobert
        _clf = clf["phobert"]
    else:
        raise ValueError

    # preprocess
    text = preprocess(text)

    # vectorize
    if embedding_type == "tfidf":
        features = vectorizer.transform(text)
    elif embedding_type == "sbert":
        features = vectorizer.encode(text)
    elif embedding_type == "phobert":
        features = phobert.encode(text)
    else:
        raise ValueError

    # classify
    pred = _clf.predict(features)

    if with_prob:
        prob = _clf.predict_proba(features)
        prob = np.max(prob, axis=1)
        return pred, prob

    return pred

#### Features

##### TF-IDF

In [242]:
def test1(text):
    pred, prob = infer(text, embedding_type="tfidf", with_prob=True)
    pprint(pred)
    pprint(lb.inverse_transform(pred))
    pprint(prob)

In [243]:
text = "em có thể diễn giải chi tiết hơn được không"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.34704396])


In [246]:
text = "hay là chút nữa liên lạc lại em nhé"
test1(text)

array([[0, 1, 0]])
array(['C2B_GLS'], dtype='<U9')
array([0.51182048])


In [248]:
text = "cảm ơn em nhưng chắc là anh không hứng thú"
test1(text)

array([[0, 0, 1]])
array(['C3_KQT'], dtype='<U9')
array([0.62048898])


In [251]:
text = "anh nghĩ là em nên dừng lại"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.46300921])


In [253]:
text = "dạ không ạ"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.46125805])


In [257]:
text = "không đâu chị ơi"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.36902174])


In [265]:
text = "ừ không, mình đang làm rồi bạn nha"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.38223752])


##### SBERT

In [244]:
def test2(text):
    pred, prob = infer(text, embedding_type="sbert", with_prob=True)
    pprint(pred)
    pprint(lb.inverse_transform(pred))
    pprint(prob)

In [245]:
text = "em có thể diễn giải chi tiết hơn được không"
test2(text)

array([[1, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.69315568])


In [247]:
text = "hay là chút nữa liên lạc lại em nhé"
test2(text)

array([[0, 1, 0]])
array(['C2B_GLS'], dtype='<U9')
array([0.95827348])


In [249]:
text = "cảm ơn em nhưng chắc là anh không hứng thú"
test2(text)

array([[0, 0, 1]])
array(['C3_KQT'], dtype='<U9')
array([0.77097445])


In [252]:
text = "anh nghĩ là em nên dừng lại"
test2(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.21407058])


In [256]:
text = "dạ không ạ"
test2(text)

array([[1, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.89481114])


In [258]:
text = "không đâu chị ơi"
test2(text)

array([[0, 0, 1]])
array(['C3_KQT'], dtype='<U9')
array([0.66558446])


In [267]:
text = "ừ không, mình đang làm rồi bạn nha"
test2(text)

array([[0, 0, 1]])
array(['C3_KQT'], dtype='<U9')
array([0.80773914])


##### PhoBERT

In [78]:
def test3(text):
    pred, prob = infer(text, embedding_type="phobert", with_prob=True)
    pprint(pred)
    pprint(lb.inverse_transform(pred))
    pprint(prob)

In [ ]:
text = "em có thể diễn giải chi tiết hơn được không"
test3(text)

In [457]:
text = "hay là chút nữa liên lạc lại em nhé"
test3(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.37629714])


# Benchmark

In [2]:
embedding_types = ["tfidf", "sbert", "phobert"]

In [3]:
class PhoBertWrapper:
    def __init__(self):
        self.model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base")
        self.tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

    def encode(self, X: Union[Text, List[Text]]):
        embeddings = None

        encoded_text = self.tokenizer(X, padding=True, truncation=True, return_tensors="pt")
        text_ids = encoded_text["input_ids"]
        with torch.no_grad():
            outputs = self.model(text_ids, output_hidden_states=True)
            last_hidden_state = outputs.hidden_states[-1]
            embeddings = last_hidden_state[:, 0, :]
            if isinstance(embeddings, torch.Tensor):
                embeddings = embeddings.detach().cpu().numpy()

        return embeddings

In [4]:
sbert = SentenceTransformer('keepitreal/vietnamese-sbert')
phobert = PhoBertWrapper()

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: keepitreal/vietnamese-sbert
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a B

## Load Data

In [5]:
botid = {
    "sunlaw": "6268f7e49f455cd4ea292d88",
    "lamhaian": "6268f8049f455c653b292e29",
    "giang.nguyen": "6268f7f89f455cd9eb292df4",
    "panasonic": "628c83a46c9e4c57fdbc5df6",
    "trung.quan": "628a14af6c9e4c08cebc3e34"
}

### Training Set

In [6]:
# load data
data = {k: load_data_from_botid(v) for k, v in botid.items()}

INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token


### Test set

## Preprocess Data

In [7]:
# preprocess
def preprocess(data: pd.DataFrame) -> pd.DataFrame:
    """ Text preprocessing
    """
    # remove duplicate
    data = data.drop_duplicates("Sentence")

    # lower
    data["Sentence"] = data["Sentence"].map(lambda x: x.lower())
    data.head(n=10)

    # word segmenation
    data["Sentence"] = data["Sentence"].map(ViTokenizer.tokenize)

    return data

In [8]:
preprocessed_data = {k: preprocess(v) for k, v in data.items()}

In [9]:
X = {k: v["Sentence"].tolist() for k, v in preprocessed_data.items()}

## Get Features (DEPRECATED)

In [7]:
""" Inititialize featurizers 
"""
tfidf = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", ngram_range=(1, 2))
sbert = SentenceTransformer('keepitreal/vietnamese-sbert')
phobert = PhoBertWrapper()

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: keepitreal/vietnamese-sbert
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a B

In [8]:
def get_features(X: Union[Text, List[Text]],
                 embedding_type: Text) -> np.ndarray:
    if embedding_type == "tfidf":
        X = tfidf.fit_transform(X).toarray()
    elif embedding_type == "sbert":
        X = sbert.encode(X)
    elif embedding_type == "phobert":
        X = phobert.encode(X)
    else:
        raise ValueError("Current supported text encoders: ['tfidf', 'sbert', 'phobert']")

    return X

In [9]:
X = {k: v["Sentence"].tolist() for k, v in preprocessed_data.items()}

In [10]:
features = {k: {t: get_features(X[k], t) for t in embedding_types} for k, v in tqdm(X.items())}

  0%|                                                                                                           | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

 33%|█████████████████████████████████                                                                  | 1/3 [00:07<00:15,  7.71s/it]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

 67%|██████████████████████████████████████████████████████████████████                                 | 2/3 [00:11<00:05,  5.63s/it]

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:17<00:00,  5.70s/it]


## Encode Labels

In [10]:
f = lambda x: [row[row == 1].index.tolist() for _, row in x.iterrows()]
y = {k: f(v) for k, v in preprocessed_data.items()}
label_encoders = {k: MultiLabelBinarizer().fit(v) for k, v in y.items()}
y = {k: label_encoders[k].transform(v) for k, v in y.items()}

## Training

In [11]:
def make_model(embs_type: Text) -> Pipeline:
    assert isinstance(embs_type, Text)

    if embs_type == "tfidf":
        pipeline = Pipeline(steps=[
            ("vect", TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", ngram_range=(1, 2))),
            ("clf", OVRStrategy(LR(max_iter=1000, n_jobs=-1), n_jobs=-1))
        ])
    elif embs_type == "sbert":
        pipeline = Pipeline(steps=[
            ("vect", FunctionTransformer(sbert.encode)),
            ("clf", OVRStrategy(LR(max_iter=1000, n_jobs=-1), n_jobs=-1))
        ])
    elif embs_type == "phobert":
        pipeline = Pipeline(steps=[
            ("vect", FunctionTransformer(phobert.encode)),
            ("clf", OVRStrategy(LR(max_iter=1000, n_jobs=-1), n_jobs=-1))
        ])
    else:
        raise ValueError("Current supported text encoders: ['tfidf', 'sbert', 'phobert']")

    return pipeline

In [12]:
models = {k: {t: make_model(t) for t in embedding_types} for k in botid.keys()}

In [13]:
pprint(models)

{'giang.nguyen': {'phobert': Pipeline(steps=[('vect',
                 FunctionTransformer(func=<bound method PhoBertWrapper.encode of <__main__.PhoBertWrapper object at 0x7fc4dc19f290>>)),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000,
                                                                  n_jobs=-1),
                                     n_jobs=-1))]),
                  'sbert': Pipeline(steps=[('vect',
                 FunctionTransformer(func=<bound method SentenceTransformer.encode of SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)>)),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000,
    

In [15]:
f1_micro = {k: {t: None for t in embedding_types} for k in botid.keys()}

In [17]:
KFOLD = 10 # https://cran.r-project.org/web/packages/cvms/vignettes/picking_the_number_of_folds_for_cross-validation.html#:~:text=When%20performing%20cross%2Dvalidation%2C%20it,common%20to%20use%2010%20folds.

# Training
for botname in tqdm(botid.keys()):
    for emb_type in embedding_types:
        model = models[botname][emb_type]
        _X = X[botname]
        _y = y[botname]
        scores = cross_validate(model, _X, _y,
                                scoring=("f1_micro"),
                                cv=KFOLD, n_jobs=1)
        f1_micro[botname][emb_type] = scores["test_score"].mean().round(2)
        # f1_macro[botname][emb_type] = scores["test_f1_macro"].mean().round(2)
        # model.fit(X[botname], y[botname])
        # models[botname][emb_type] = model

  0%|                                                                                                                 | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [06:25<00:00, 77.03s/it]


In [18]:
f1_micro

{'sunlaw': {'tfidf': 0.65, 'sbert': 0.81, 'phobert': 0.59},
 'lamhaian': {'tfidf': 0.23, 'sbert': 0.39, 'phobert': 0.22},
 'giang.nguyen': {'tfidf': 0.26, 'sbert': 0.48, 'phobert': 0.31},
 'panasonic': {'tfidf': 0.35, 'sbert': 0.63, 'phobert': 0.3},
 'trung.quan': {'tfidf': 0.69, 'sbert': 0.81, 'phobert': 0.68}}

### Visualize

In [35]:
# Ref: https://docs.bokeh.org/en/latest/docs/user_guide/categorical.html#visual-offset

output_notebook()
# output_file("./dodged_bars.html")

botnames = [botname for botname in botid.keys()]
vis_tfidf = [f1_micro[botname]["tfidf"] for botname in botnames]
vis_sbert = [f1_micro[botname]["sbert"] for botname in botnames]
vis_phobert = [f1_micro[botname]["phobert"] for botname in botnames]

vis_data = {
    "botnames": botnames,
    "tfidf": vis_tfidf,
    "sbert": vis_sbert,
    "phobert": vis_phobert
}

source = ColumnDataSource(data=vis_data)

TITLE = "Models Benchmark (w/ diff vectorizer)"
p = figure(x_range=botnames, y_range=(0, 1), height=40,
           title=TITLE, toolbar_location=None, tools="")

p.vbar(x=dodge("botnames", -0.25, range=p.x_range),
       top="tfidf", width=0.2, source=source,
       color="#c9d9d3", legend_label="TF-IDF (Baseline)")

p.vbar(x=dodge("botnames", 0.0, range=p.x_range),
       top="sbert", width=0.2, source=source,
       color="#718dbf", legend_label="Vietnamese SBERT (No FT)")

p.vbar(x=dodge("botnames", 0.25, range=p.x_range),
       top="phobert", width=0.2, source=source,
       color="#e84d60", legend_label="PhoBERT-base (No FT)")

p.x_range.range_padding = 0.1
p.xgrid.grid_line_color = None
p.yaxis.axis_label = "f1 micro"
p.legend.orientation = "vertical"
p.legend.location = "top_center"

show(p)

# p.background_fill_color = None
# p.border_fill_color = None
# export_png(p, filename="../imgs/emb_benchmark.png")

Loading BokehJS ...

## Evaluate